In [ ]:
%%capture
!pip install -q opendatasets
!pip install ydata_profiling
!pip install sentence_transformers

In [ ]:

import torch
import time
import warnings
import numpy as np
import opendatasets as od
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import torch.nn as nn

from ydata_profiling import ProfileReport
from collections import defaultdict
from itertools import chain
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from transformers import Adafactor, AdamW, get_linear_schedule_with_warmup
warnings.filterwarnings('ignore')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


instruções login API kaggle: http://bit.ly/kaggle-creds

user: lucasmarchiori

key: 39d6c820afd2569227729bad72643756


vantagem da API é que não precisa ficar fazendo upload do arquivo, ela ja baixa direto no colab sempre q executar o código

In [ ]:
od.download('https://www.kaggle.com/datasets/kashishparmar02/social-media-sentiments-analysis-dataset/data') # insert your kaggle  username and key
df = pd.read_csv('/content/social-media-sentiments-analysis-dataset/sentimentdataset.csv')


Skipping, found downloaded files in "./social-media-sentiments-analysis-dataset" (use force=True to force download)


In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [ ]:
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True) # excluindo colunas desnecessarias
df['Sentiment'] = df['Sentiment'].str.strip() # como nosso target é texto, pode ser q eles tenham espaços a mais, o strip remove todo espaço antes e após a string

In [ ]:
ProfileReport(df)

In [ ]:
# Selecionando classes q aparecem menos de 5 vezes
not_important = df['Sentiment'].value_counts()[df['Sentiment'].value_counts() < 5].index

# Atribuindo a classificação das classes q aparecem menos de 5 vezes como Others'
df.loc[df['Sentiment'].isin(not_important), 'Sentiment'] = 'Others'


In [ ]:
df['Sentiment'].value_counts()

Others           274
Positive          45
Joy               44
Excitement        37
Contentment       19
Neutral           18
Gratitude         18
Curiosity         16
Serenity          15
Happy             14
Despair           11
Nostalgia         11
Loneliness         9
Sad                9
Hopeful            9
Awe                9
Grief              9
Embarrassed        8
Confusion          8
Acceptance         8
Enthusiasm         7
Pride              7
Elation            7
Euphoria           7
Determination      7
Numbness           6
Regret             6
Melancholy         6
Surprise           6
Ambivalence        6
Bad                6
Frustration        6
Playful            6
Indifference       6
Hate               6
Inspiration        6
Happiness          5
Hope               5
Disgust            5
Betrayal           5
Frustrated         5
Bitterness         5
Empowerment        5
Inspired           5
Name: Sentiment, dtype: int64

In [ ]:
df['Sentiment'].unique()

array(['Positive', 'Others', 'Neutral', 'Disgust', 'Happiness', 'Joy',
       'Awe', 'Surprise', 'Acceptance', 'Confusion', 'Excitement',
       'Pride', 'Elation', 'Euphoria', 'Contentment', 'Serenity',
       'Gratitude', 'Hope', 'Empowerment', 'Enthusiasm', 'Despair',
       'Grief', 'Loneliness', 'Frustration', 'Regret', 'Curiosity',
       'Indifference', 'Numbness', 'Melancholy', 'Nostalgia',
       'Ambivalence', 'Determination', 'Hopeful', 'Playful', 'Inspired',
       'Bitterness', 'Frustrated', 'Inspiration', 'Betrayal',
       'Embarrassed', 'Sad', 'Hate', 'Bad', 'Happy'], dtype=object)

In [ ]:
class_to_int = {
'Positive':0,
'Others':1,
'Neutral':2,
'Disgust':3,
'Happiness':4,
'Joy':5,
'Awe':6,
'Surprise':7,
'Acceptance':8,
'Confusion':9,
'Excitement':10,
'Pride':11,
'Elation':12,
'Euphoria':13,
'Contentment':14,
'Serenity':15,
'Gratitude':16,
'Hope':17,
'Empowerment':18,
'Enthusiasm':19,
'Despair':20,
'Grief':21,
'Loneliness':22,
'Frustration':23,
'Regret':24,
'Curiosity':25,
'Indifference':26,
'Numbness':27,
'Melancholy':28,
'Nostalgia':29,
'Ambivalence':30,
'Determination':31,
'Hopeful':32,
'Playful':33,
'Inspired':34,
'Bitterness':35,
'Frustrated':36,
'Inspiration':37,
'Betrayal':38,
'Embarrassed':39,
'Sad':40,
'Hate':41,
'Bad':42,
'Happy':43

}

# Replace the class names in the DataFrame
df['Label'] = df['Sentiment'].replace(class_to_int)


In [ ]:
fig = px.bar(x=df['Sentiment'].unique(), y=df['Sentiment'].value_counts(sort=False),
             color= df['Sentiment'].unique(),
             labels={'y':'Frequência', 'x':'Sentimento'})
fig.show()

In [ ]:
text_size = df['Text'].apply(len)
fig = px.histogram(x=text_size)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
MODEL = f"bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(MODEL)

In [ ]:
df = df[['Text', 'Label']]
train_df_, val_df = train_test_split(df, test_size=0.35,  random_state=42)
train_df, test_df = train_test_split(train_df_, test_size=0.60, random_state=42)

In [ ]:
tokenized_train = tokenizer.batch_encode_plus(train_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                               max_length = 480, return_tensors = 'pt')
tokenized_val = tokenizer.batch_encode_plus(val_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                            max_length = 480, return_tensors = 'pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
input_ids_train = tokenized_train['input_ids']
attention_masks_train = tokenized_train['attention_mask']
labels_train = torch.tensor(train_df.Label.values)

input_ids_val = tokenized_val['input_ids']
attention_masks_val = tokenized_val['attention_mask']
labels_val = torch.tensor(val_df.Label.values.tolist())

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=6)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=6)

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, labels):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(MODEL)
    self.drop = nn.Dropout(p=0.2)
    self.out = nn.Linear(self.bert.config.hidden_size, labels)

  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
    output = self.drop(pooled_output)
    return self.out(output)

model = SentimentClassifier(44)
model = model.to(device)

In [ ]:
# documentação para mudança de parametros ou otimizador: https://huggingface.co/docs/transformers/main_classes/optimizer_schedules
# Tentei utilizar Adafactor porém todos accuracia ficaram extremamente baixas,sendo a maior 0.26, tentarei com AdamW para ver
EPOCHS = 10
total_steps = len(dataloader_train) * EPOCHS
#optimizer = Adafactor(model.parameters(), lr = 2e-5, relative_step= False)
optimizer = AdamW(model.parameters(), lr = 2e-5, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
def eval_model(model, dataloader, device, criterion, epoch=0):

    model = model.train()
    correct_pred = 0
    losses = []
    predictions = []
    true_values = []

    data_loader = tqdm(dataloader, desc=f"Epoch {epoch}", leave=False, disable=False)

    for d in dataloader:
        input_ids = d[0].to(device)
        attention_mask = d[1].to(device)
        targets = d[2].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        correct_pred += torch.sum(preds == targets)

        loss = criterion(outputs, targets)
        losses.append(loss.item())
        loss.backward()


        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

       data_loader.set_postfix(loss=np.mean(losses))
        predictions.append(preds.cpu())
        true_values.append(targets.cpu())


    predictions = np.concatenate(predictions, axis=0)
    true_values = np.concatenate(true_values, axis=0)


    accuracy = correct_pred.item() / len(dataloader)

    return (predictions, true_values)

In [ ]:
def train_model(model, dataloader, optimizer, device, scheduler, criterion, epoch=0):

    model = model.train()

    losses = []

    data_loader = tqdm(dataloader, desc=f"Epoch {epoch}", leave=False, disable=False)

    for d in dataloader:
        input_ids = d[0].to(device)
        attention_mask = d[1].to(device)
        targets = d[2].to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask)


        #_, preds = torch.max(outputs, dim=1)

        loss = criterion(outputs, targets)
        losses.append(loss.item())
        loss.backward()


        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

=        data_loader.set_postfix({'training_loss': '{:.3f}'.format(np.mean(losses))})



In [ ]:
criterion = nn.CrossEntropyLoss().to(device)

best = 0
for epoch in range(EPOCHS):
  train_model(model, dataloader_train, optimizer, device, scheduler, criterion, epoch+1)
  prediction, true_values = eval_model(model, dataloader_validation, device, criterion, epoch +1)
  accuracy = accuracy_score(prediction, true_values)
  if accuracy > best:
    torch.save(model.state_dict(), 'best_model_others.bin')
    best = accuracy
print(f'A melhor accuracia foi: {best}')

A melhor accuracia foi: 0.5486381322957199


In [ ]:
model.load_state_dict(torch.load('best_model_others.bin'))
model = model.to(device)
prediction, true_values = eval_model(model, dataloader_validation, device, criterion)

In [ ]:
print(classification_report(prediction,true_values))

              precision    recall  f1-score   support

           0       0.95      0.67      0.78        27
           1       0.98      0.88      0.93       106
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       1.00      0.26      0.41        66
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.25      0.17      0.20         6
           9       0.60      0.50      0.55         6
          10       0.00      0.00      0.00         3
          12       0.75      0.60      0.67         5
          13       0.50      1.00      0.67         2
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
          17       1.00      1.00      1.00         1
          18       0.00    

como podemos ver acima, as classes que continham poucos valores o mesmo não acertou nada, tendo um precision, recall e f1 de zero, na proxima seção iremos ver o que acontece quando ignoramos a categoria others e fazemos com todos os labels

# Sem a categoria OTHERS

In [ ]:
df = pd.read_csv('/content/social-media-sentiments-analysis-dataset/sentimentdataset.csv')
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True) # excluindo colunas desnecessarias
df['Sentiment'] = df['Sentiment'].str.strip() # como nosso target é texto, pode ser q eles tenham espaços a mais, o strip remove todo espaço antes e após a string
df['Sentiment'].value_counts()

Positive          45
Joy               44
Excitement        37
Contentment       19
Neutral           18
                  ..
LostLove           1
EmotionalStorm     1
Suffering          1
Bittersweet        1
Intrigue           1
Name: Sentiment, Length: 191, dtype: int64

In [ ]:
df['Sentiment'].unique()

array(['Positive', 'Negative', 'Neutral', 'Anger', 'Fear', 'Sadness',
       'Disgust', 'Happiness', 'Joy', 'Love', 'Amusement', 'Enjoyment',
       'Admiration', 'Affection', 'Awe', 'Disappointed', 'Surprise',
       'Acceptance', 'Adoration', 'Anticipation', 'Bitter', 'Calmness',
       'Confusion', 'Excitement', 'Kind', 'Pride', 'Shame', 'Elation',
       'Euphoria', 'Contentment', 'Serenity', 'Gratitude', 'Hope',
       'Empowerment', 'Compassion', 'Tenderness', 'Arousal', 'Enthusiasm',
       'Fulfillment', 'Reverence', 'Despair', 'Grief', 'Loneliness',
       'Jealousy', 'Resentment', 'Frustration', 'Boredom', 'Anxiety',
       'Intimidation', 'Helplessness', 'Envy', 'Regret', 'Curiosity',
       'Indifference', 'Numbness', 'Melancholy', 'Nostalgia',
       'Ambivalence', 'Determination', 'Zest', 'Hopeful', 'Proud',
       'Grateful', 'Empathetic', 'Compassionate', 'Playful',
       'Free-spirited', 'Inspired', 'Confident', 'Bitterness', 'Yearning',
       'Fearful', 'Apprehensiv

In [ ]:
class_to_int = {}
value = 0
for i in df['Sentiment'].unique():
    class_to_int[i] = value
    value += 1
df['Label'] = df['Sentiment'].replace(class_to_int)

In [ ]:
fig = px.bar(x=df['Sentiment'].unique(), y=df['Sentiment'].value_counts(sort=False),
             color= df['Sentiment'].unique(),
             labels={'y':'Frequência', 'x':'Sentimento'})
fig.show()

In [ ]:
text_size = df['Text'].apply(len)
fig = px.histogram(x=text_size)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
df = df[['Text', 'Label']]
train_df_, val_df = train_test_split(df, test_size=0.35,  random_state=42)
train_df, test_df = train_test_split(train_df_, test_size=0.60, random_state=42)

In [ ]:
tokenized_train = tokenizer.batch_encode_plus(train_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                              max_length = 480, return_tensors = 'pt')
tokenized_val = tokenizer.batch_encode_plus(val_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                            max_length = 480, return_tensors = 'pt')

In [ ]:
input_ids_train = tokenized_train['input_ids']
attention_masks_train = tokenized_train['attention_mask']
labels_train = torch.tensor(train_df.Label.values)

input_ids_val = tokenized_val['input_ids']
attention_masks_val = tokenized_val['attention_mask']
labels_val = torch.tensor(val_df.Label.values.tolist())

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=6)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=6)

In [ ]:
model = SentimentClassifier(len(class_to_int))
model = model.to(device)

In [ ]:
best = 0
for epoch in range(EPOCHS):
  train_model(model, dataloader_train, optimizer, device, scheduler, criterion, epoch+1)
  prediction, true_values = eval_model(model, dataloader_validation, device, criterion, epoch +1)
  accuracy = accuracy_score(prediction, true_values)
  if accuracy > best:
    torch.save(model.state_dict(), 'best_model_all.bin')
    best = accuracy
print(f'A melhor accuracia foi: {best}')

A melhor accuracia foi: 0.007782101167315175


In [ ]:
model.load_state_dict(torch.load('best_model_all.bin'))
model = model.to(device)
prediction, true_values = eval_model(model, dataloader_validation, device, criterion)

In [ ]:
print(classification_report(prediction,true_values))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.33      0.03      0.05        37
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         0
          19       0.00    

Percebemos que muitas classes apenas diminuiu mais a nossa acuracia para zero, em nosso proximo teste iremos filtrar, para todas as classes com menos de 15 aparições ser removidas, ao inves de considerada como others

# Apenas 15 ou mais aparições da classe, sem a classe Others

In [ ]:
df = pd.read_csv('/content/social-media-sentiments-analysis-dataset/sentimentdataset.csv')
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True) # excluindo colunas desnecessarias
df['Sentiment'] = df['Sentiment'].str.strip() # como nosso target é texto, pode ser q eles tenham espaços a mais, o strip remove todo espaço antes e após a string
# Selecionando classes q aparecem menos de 15 vezes
not_important = df['Sentiment'].value_counts()[df['Sentiment'].value_counts() < 15].index
# Atribuindo a classificação das classes q aparecem menos de 15 vezes como Others'
df.loc[df['Sentiment'].isin(not_important), 'Sentiment'] = 'Others'
df = df[df.Sentiment != 'Others']
df['Sentiment'].value_counts()

Positive       45
Joy            44
Excitement     37
Contentment    19
Neutral        18
Gratitude      18
Curiosity      16
Serenity       15
Name: Sentiment, dtype: int64

In [ ]:
class_to_int = {}
value = 0
for i in df['Sentiment'].unique():
    class_to_int[i] = value
    value += 1
df['Label'] = df['Sentiment'].replace(class_to_int)

In [ ]:
df = df[['Text', 'Label']]
train_df_, val_df = train_test_split(df, test_size=0.35,  random_state=42)
train_df, test_df = train_test_split(train_df_, test_size=0.60, random_state=42)

In [ ]:
tokenized_train = tokenizer.batch_encode_plus(train_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                              max_length = 480, return_tensors = 'pt')
tokenized_val = tokenizer.batch_encode_plus(val_df.Text.values.tolist(), add_special_tokens = True, return_attention_mask = True, pad_to_max_length = True,
                                            max_length = 480, return_tensors = 'pt')

In [ ]:
input_ids_train = tokenized_train['input_ids']
attention_masks_train = tokenized_train['attention_mask']
labels_train = torch.tensor(train_df.Label.values)

input_ids_val = tokenized_val['input_ids']
attention_masks_val = tokenized_val['attention_mask']
labels_val = torch.tensor(val_df.Label.values.tolist())

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=6)
dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=6)

In [ ]:
model = SentimentClassifier(len(class_to_int))
model = model.to(device)

In [ ]:
best = 0
for epoch in range(EPOCHS):
  train_model(model, dataloader_train, optimizer, device, scheduler, criterion, epoch+1)
  prediction, true_values = eval_model(model, dataloader_validation, device, criterion, epoch +1)
  accuracy = accuracy_score(prediction, true_values)
  if accuracy > best:
    torch.save(model.state_dict(), 'best_model_five.bin')
    best = accuracy
print(f'A melhor accuracia foi: {best}')

A melhor accuracia foi: 0.25333333333333335


In [ ]:
model.load_state_dict(torch.load('best_model_five.bin'))
model = model.to(device)
prediction, true_values = eval_model(model, dataloader_validation, device, criterion)
print(classification_report(prediction,true_values))

              precision    recall  f1-score   support

           0       0.75      0.25      0.38        48
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         0

    accuracy                           0.16        75
   macro avg       0.09      0.03      0.05        75
weighted avg       0.48      0.16      0.24        75



Temos que o primeiro método se saiu melhor em geral, atribuindo a categoria others para labels que aparecem pouco em nosso dataset

# Test Dataset

Aqui eremos utilizar nosso transformers que se saiu melhor em nosso dataset de teste

In [ ]:
model.load_state_dict(torch.load('best_model_others.bin'))
model = model.to(device)
prediction, true_values = eval_model(model, dataloader_train, device, criterion)
print(classification_report(prediction,true_values))